In [1]:
%run ./read_file.ipynb 

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-d18c5a14-0339-4921-a175-df862a3e23dd;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (99ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (35ms)
:: resolution report :: resolve 837ms :: artifacts dl 136ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	-------

23/03/09 01:52:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/09 01:52:05 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [39]:
def build_adjaceny(df, dataset = 'ecar'):
    
    start_time = time.time()
    
    if dataset == 'labels':
        df = df.sort(col("ts2").asc())
    else : 
        #first step sort the data and check
        df = df.sort(col("timestamp").asc())
    
    #now get overlapping connections

    #build collections of each type of ID
    #TODO make this one select statement
    col_objID = df.select("objectID").distinct().collect()
    col_actID = df.select("actorID").distinct().collect()
    
    print(len(col_actID))

    #put these columns into a dictionary and turn into a spark data frame
    data = [{"actorIDs" : [row.actorID for row in col_actID], \
                   "objectIDs" : [row.objectID for row in col_objID]}]

    id_intersect = spark.createDataFrame(data)

    #compare ID pools to find actIDs that are nodes
    id_intersect = id_intersect.withColumn("id_int", \
                                           array_intersect(col("actorIDs"),col("objectIDs")))

    #now build a list of the nodal actIDs and print the length
    actIDs = [row.id_int for row in id_intersect.select("id_int").collect()][0]
    
    print("length actIDs: "+str(len(actIDs)))
    print("elapsed time: "+ str(time.time() - start_time))
    #create ref dict
    dict_node_edge_events = {}
    count = 0 
    
    #for all actors in actIDs
    for aID in actIDs: 
        
        birth_event = df.where((col("objectID") == aID) \
                               & (col("actorID")!= aID)).limit(1).collect()
        
        if len(birth_event) > 0: 
            first_id = birth_event[0]['id']

            #now add this to the dictionary as a key
            first_event = str(aID+':'+first_id)
            dict_node_edge_events[first_event] = []

            #now add all the events where the actorID is present but not the objectID, group by unique object ID
            #and pull out the first event - and I checked and believe the sorting holds through grouping
            col_select_children_events = df.select('objectID','id')\
            .where((col("actorID") == aID) & \
                     (col("objectID") != aID)).groupBy("objectID").agg(first("id")).collect() 

            #add children objects into dictioinary 
            dict_node_edge_events[first_event] = [str(row["objectID"]+':'+row["first(id)"]) for row in col_select_children_events]
        else:
            count += 1
    
    print("no birth event for " + str(count)+" events.")
    
    # create a list of tuples from the dictionary
    data_list = list(dict_node_edge_events.items())

    # convert the list of tuples to a PySpark RDD
    data_rdd = spark.sparkContext.parallelize(data_tuples)

    # create a PySpark DataFrame from the RDD
    df_final = data_rdd.toDF()
    
    #explode list into column, keeping position for some sort of rank just in case
    df_final_exp = df_final.select("_1",posexplode("_2"))
    df_final_exp_col = df_final_exp.selectExpr("_1 as parent", "pos as order", "col as child")
    
    print("elapsed time 2: "+ str(time.time() - start_time))
    return df_final_exp_col

In [ ]:
df_labels = readCheckpoint_bcm('labels')
#create a timestamp column and drop the old timestamp - does not appear necessary
df_labels = df_labels.withColumn('ts2', to_timestamp(col('timestamp'))).drop('timestamp')
print('event count' + str(df_labels.count()))
#check new col data types
df_labels.printSchema()

In [40]:
#run code with labels dataframe
df_labels_adj = build_adjaceny(df_labels, 'labels')

150
length actIDs: 90
elapsed time: 1.3366224765777588
no birth event for 0 events.
elapsed time 2: 25.059102296829224


In [41]:
df_labels_adj.limit(5).show()

+--------------------+-----+--------------------+
|              parent|order|               child|
+--------------------+-----+--------------------+
|d5846376-9596-428...|    0|000401b2-4413-4f2...|
|d5846376-9596-428...|    1|00263384-6151-407...|
|d5846376-9596-428...|    2|0093031c-0521-4e7...|
|d5846376-9596-428...|    3|00979bf4-65da-491...|
|d5846376-9596-428...|    4|00accb8e-39bf-40c...|
+--------------------+-----+--------------------+



In [42]:
df_labels_adj.limit(1).collect()[0]['parent']

'd5846376-9596-4283-8505-8e6112abf1f2:e15537fe-fdb0-49d4-a1fd-de34e8215617'